# Our heuristic: generation, comparison and scoring functions

In [1]:
import random
import numpy as np
import xpress as xp
from collections import defaultdict

def elements_gen(N_items):
    volumes = defaultdict(dict)
    demand = defaultdict(dict)
    costs = defaultdict(dict)
    alphas = defaultdict(dict)
    price = defaultdict(dict)

    for i in range(1, N_items + 1):
        volumes[i] = float("{:.2f}".format(random.uniform(0.1, 10)))  # Random volume for item i
        demand[i] = int(random.randint(1, 16))  # Random demand for item i
        costs[i] = int(random.uniform(1.0, 101.0))  # Random cost for item i
        price[i] = 1.5 * costs[i]

        for j in range(1, N_items + 1):
            if i == j:
                alphas[(j, i)] = 0.0
            else:
                alphas[(j, i)] = float("{:.2f}".format(random.uniform(0, 0.9)))  # Random alpha for item j -> item i

    return volumes, demand, costs, alphas, price



def optimization_model(set_items):

    volumes = set_items[0]
    demand = set_items[1]
    costs = set_items[2]
    alpha_m = set_items[3]
    price = set_items[4]

    keys_list = list(volumes.keys())
    
    # Create the problem
    prob = xp.problem()
    # Decision variables

    max_row = max(list(demand.keys()))
    print(max_row)
    key_ind = list(range(max_row+1))


    x = np.array([xp.var(vartype=xp.binary, name=f'x_{i}') for i in key_ind])
    u = np.array([xp.var(vartype=xp.integer, lb=0, name=f'u_{i}') for i in key_ind])
    z = np.array([[xp.var(vartype=xp.binary, name=f'z_{i}_{j}') for j in key_ind] for i in key_ind])

    prob.addVariable(x, u, z)

   
    d = np.zeros(max_row+1)
    test = np.zeros(max_row+1)
    w = np.zeros(max_row+1)
    c = np.zeros(max_row+1)
    alpha = np.zeros((max_row+1, max_row+1))

    for i, value in demand.items():
        d[i] = value
        test[i] = 1
    for i, value in volumes.items():
        w[i] = value
    for i, value in costs.items():
        c[i] = value

    for (i, j), value in alpha_m.items():
        alpha[i, j] = value
    p = 1.5 * c

    C = 100* np.dot(w, d)  
    M = 1000000000
    K_T = 100
    delta = 0.7
    
    key_index = [x for x in key_ind if test[x] != 0]
    print(key_index)
        # Objective function
    prob.setObjective(xp.Sum(K_T * p[i] * d[i] * x[i] for i in key_index) +xp.Sum(K_T * p[i] * xp.Sum(
    delta*alpha[j, i] * d[j] * (x[i] - z[i, j]) for j in key_index if j != i)for i in key_index)
    - xp.Sum(c[i] * u[i] for i in key_index),sense=xp.maximize)
    print('passed')

    # Constraints
    prob.addConstraint(xp.Sum(w[i] * u[i] for i in key_index) <= C)
    for i in key_index:
        prob.addConstraint(u[i] <= M * x[i])
        prob.addConstraint(
            u[i]>= K_T * d[i] * x[i]+ K_T*xp.Sum(delta*alpha[j, i] * d[j] * (x[i] - z[i, j]) for j in key_index if j != i) )
        for j in key_index:
            if j != i:
                prob.addConstraint(z[i, j] <= x[i])
                prob.addConstraint(z[i, j] <= x[j])
                prob.addConstraint(z[i, j] >= x[i] + x[j] - 1)
                prob.addConstraint(z[j, i] == z[i, j]) 
    prob.addConstraint(xp.Sum(u[i] for i in key_index) == K_T * xp.Sum(d[i] for i in key_index))

    # Solve the problem
    prob.solve()

    optimal_x = {i: prob.getSolution(x[i]) for i in key_index}
    optimal_u = {i: prob.getSolution(u[i]) for i in key_index}
    non_zero_optimal_x = {i: value for i, value in optimal_x.items() if value != 0}
    non_zero_optimal_u = {i: value for i, value in optimal_u.items() if value != 0}

    optimal_objective_value = prob.getObjVal()
    return(optimal_objective_value, optimal_x, optimal_u)



def divide_set_1000(initial_set):


    divided_sets= []
    divided = []
    for i in range(len(initial_set)):
        if i != 3:
            divided_dicts = []
            my_dict = initial_set[i]
            items = list(my_dict.items())
        
            # Calculate the number of elements in each smaller dictionary
            elements_per_dict = len(my_dict) // 10

            # Iterate through the range of 0 to 10 (exclusive) with a step of 100
            for i in range(0, len(my_dict), elements_per_dict):
                # Extract a slice of elements from the main list and convert it back to a dictionary
                divided_dict = dict(items[i: i + elements_per_dict])
                divided_dicts.append(divided_dict)
            divided.append(divided_dicts)
        else:
            my_dict = initial_set[3]
            divided_alphas = []
            for ele in divided[0]:
                indices_ephem = list(ele.keys())
                dict_alpha = {}
                for j in indices_ephem:
                    for k in indices_ephem:
                        dict_alpha[(j,k)] = my_dict[(j,k)]
                divided_alphas.append(dict_alpha)
            divided.append(divided_alphas)

    
    for i in range(len(divided[0])):
        helper = []
        for ele in divided:
            helper.append(ele[i])
        divided_sets.append(helper)
    return(divided_sets)



def divide_sets_100(seti_items):
    # Initialize two dictionaries to hold the elements
    first_set_50 = []
    second_set_50 = []

    # Iterate through the dictionary items and split them into dictionaries

    for i in range(len(seti_items)):
        if i != 3:
            count = 0
            first_dict = {}
            second_dict = {}
            for key, value in seti_items[i].items():
                if count < 50:
                    first_dict[key] = value
                else:
                    second_dict[key] = value
                count += 1
            first_set_50.append(first_dict)
            second_set_50.append(second_dict)
        else: 
            keys_alp_1 = list(first_set_50[0].keys())
            keys_alp_2 = list(second_set_50[0].keys())
            first_alp = {}
            second_alp = {}
            for j in keys_alp_1:
                for k in keys_alp_1:
                    first_alp[(j,k)] = seti_items[3][(j,k)]
            for j in keys_alp_2:
                for k in keys_alp_2:
                    second_alp[(j,k)] = seti_items[3][(j,k)]  
            first_set_50.append(first_alp)
            second_set_50.append(second_alp)                             

    return(first_set_50, second_set_50)



def calculate_score(seti_items):
    u = optimization_model(seti_items)[2]
    max_u = max(list(u.values()))
    calculated_scores = {i: u[i] / max_u for i in list(u.keys())}
    # Sort the dictionary items by values in descending order
    sorted_scores = sorted(calculated_scores.items(), key=lambda x: x[1], reverse=True)
    return {k: v for k, v in sorted_scores}



def divide_set_by_scores(ranked_set_items, N_by_set):
    # Initialize two dictionaries to hold the elements
    first_set_N = []
    second_set_N = []

    # Iterate through the dictionary items and split them into dictionaries

    for i in range(len(ranked_set_items)):
        if i != 3:
            count = 0
            first_dict = {}
            second_dict = {}
            for key, value in ranked_set_items[i].items():
                if count < N_by_set:
                    first_dict[key] = value
                else:
                    second_dict[key] = value
                count += 1
            first_set_N.append(first_dict)
            second_set_N.append(second_dict)
        else:
            keys_N_alp_1 = list(first_set_N[0].keys())
            keys_N_alp_2 = list(second_set_N[0].keys())
            first_N_alp = {}
            second_N_alp = {}
            for j in keys_N_alp_1:
                for k in keys_N_alp_1:
                    first_N_alp[(j,k)] = ranked_set_items[3][(j,k)]
            for j in keys_N_alp_2:
                for k in keys_N_alp_2:
                    second_N_alp[(j,k)] = ranked_set_items[3][(j,k)]  
            first_set_N.append(first_N_alp)
            second_set_N.append(second_N_alp)           
    return(first_set_N, second_set_N)


# Data generation

In [2]:
N_elements = 3000
volumes, demand, costs, alpha, price = elements_gen(N_elements)
random_index = random.sample(range(1, N_elements + 1), 1000)
volumes_selec = {}
demand_selec = {}
costs_selec = {}
alpha_selec = {}
price_selec = {}
index = []
for i in random_index:
    volumes_selec[i] = volumes[i]
    demand_selec[i] = demand[i]
    costs_selec[i] = costs[i]
    for j in random_index:
        index.append((i,j))
        alpha_selec[(i,j)] = alpha[(i,j)]
    price_selec[i] = price[i]
set_items = [volumes_selec, demand_selec, costs_selec, alpha_selec, price_selec]


# Saving data

In [138]:
import pandas as pd

# Define an empty DataFrame with column names
df_initial = pd.DataFrame(columns=['Product', 'Volumes', 'Demand', 'Costs', 'Alphas', 'Price'])
ind = list(range(1, 3001))
product_index = ind.copy()

df_initial['Product'] = product_index
df_initial['Volumes'] = volumes.values()
df_initial['Demand'] = demand.values()
df_initial['Costs'] = costs.values()
df_initial['Price'] = price.values()

alpha_li = []
for i in range(1, len(volumes)+1):
    alpha_i = []
    for j in range(1, len(volumes)+1):
        alpha_i.append(alpha[(i,j)])
    alpha_li.append(alpha_i)
df_initial['Alphas'] = alpha_li
df_initial.to_csv('data_scoring_initial.csv', index=False)

In [116]:
# Define an empty DataFrame with column names
df_scoring = pd.DataFrame(columns=['Product_scoring', 'Volumes_scoring', 'Demand_scoring', 'Costs_scoring', 'Alphas_scoring', 'Price_scoring'])
ind = list(range(1, 1001))

df_scoring['Product_scoring'] = random_index
df_scoring['Volumes_scoring'] = volumes_selec
df_scoring['Demand_scoring'] = demand_selec
df_scoring['Costs_scoring'] = costs_selec
df_scoring['Price_scoring'] = price_selec

alpha_li = []
for i in range(1, len(volumes_selec)+1):
    alpha_i = []
    for j in range(1, len(volumes_selec)+1):
        alpha_i.append(alpha[(i,j)])
    alpha_li.append(alpha_i)
df_scoring['Alphas'] = alpha_li
df_scoring.to_csv('data_scoring_1000.csv', index=False)

# The scoring algorithm: dividing into subsets

In [3]:
Set_100 = divide_set_1000(set_items)

In [4]:
scores_f = []
Set_100 = divide_set_1000(set_items)
Sets_50 = []
Sets_50_sorted = []

for st in Set_100:
    a = divide_sets_100(st)
    Sets_50.append(a)

for ele in Sets_50:
    medi = []
    scr = []
    for st_50 in ele:
        a = calculate_score(st_50)
        scr.append(a)
        d = {}
        w = {}
        p = {}
        c = {}
        alphas = {}
        my_set_hp = []
        for ind in a.keys():
            d[ind] = set_items[1][ind]
            w[ind] = set_items[0][ind]
            c[ind] = set_items[2][ind]
            for j in a.keys():
                alphas[(ind,j)] = set_items[3][(ind,j)]
            p[ind] = set_items[4][ind]
        my_set_hp.append(w)
        my_set_hp.append(d)
        my_set_hp.append(c)
        my_set_hp.append(alphas)
        my_set_hp.append(p)
        medi.append(divide_set_by_scores(my_set_hp, 25))
    scores_f.append(scr)
    Sets_50_sorted.append(medi)


Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('C:/xpressmp/bin/xpauth.xpr')
2992
[5, 35, 53, 238, 341, 347, 451, 473, 485, 586, 734, 756, 766, 782, 868, 898, 919, 1030, 1058, 1150, 1278, 1300, 1642, 1652, 1725, 1807, 1813, 1896, 1911, 1961, 2004, 2125, 2249, 2290, 2372, 2458, 2472, 2496, 2513, 2552, 2621, 2629, 2644, 2675, 2700, 2736, 2788, 2791, 2827, 2992]
passed
FICO Xpress v9.0.0, Hyper, solve started 16:38:23, Jul 28, 2023
Heap usage: 2212MB (peak 2212MB, 99MB system)
Maximizing MILP noname using up to 8 threads and up to 63GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
      9902 rows      8964035 cols        24790 elements   8964035 entities
Presolved problem has:
      3777 rows         1325 cols        11315 elements      1325 entities
LP relaxation tightened
Presolve finished in 12 seconds
Heap us

# Merged lists: 10 instances each one of 4 sets of 50 elements

In [33]:
merged_sets_50 = []
for ele in Sets_50_sorted:
    midi = []
    hp1 = []
    hp2 = []
    hp3 = []
    hp4 = []
    for r in range(len(Sets_50_sorted[0][0][0])):
        if r != 3:
            a = {**ele[0][0][r], **ele[1][0][r]}
            index_merg = list(a.keys())
            hp1.append(a)
        else: 
            a = {}
            for ind_i in index_merg:
                for ind_j in index_merg:
                    a[(ind_i, ind_j)] = set_items[3][(ind_i, ind_j)] 
            hp1.append(a)
    midi.append(hp1)
    for r in range(len(Sets_50_sorted[0][0][0])):
        if r != 3:
            a = {**ele[0][0][r], **ele[1][1][r]}
            index_merg = list(a.keys())
            hp2.append(a)
        else: 
            a = {}
            for ind_i in index_merg:
                for ind_j in index_merg:
                    a[(ind_i, ind_j)] = set_items[3][(ind_i, ind_j)] 
            hp2.append(a)
    midi.append(hp2)
    for r in range(len(Sets_50_sorted[0][0][0])):
        if r != 3:
            a = {**ele[0][1][r], **ele[1][0][r]}
            index_merg = list(a.keys())
            hp3.append(a)
        else: 
            a = {}
            for ind_i in index_merg:
                for ind_j in index_merg:
                    a[(ind_i, ind_j)] = set_items[3][(ind_i, ind_j)] 
            hp3.append(a)
    midi.append(hp3)
    for r in range(len(Sets_50_sorted[0][0][0])):
        if r != 3:
            a = {**ele[0][1][r], **ele[1][1][r]}
            index_merg = list(a.keys())
            hp4.append(a)
        else: 
            a = {}
            for ind_i in index_merg:
                for ind_j in index_merg:
                    a[(ind_i, ind_j)] = set_items[3][(ind_i, ind_j)] 
            hp4.append(a)
    midi.append(hp4)

    merged_sets_50.append(midi)


# 4 sets to enter to the model (refer to the report for the implementation explanation)

In [40]:
before_model_50 = []
for ele in Sets_50_sorted:
    midib = []
    hlpb1 = []
    hlpb2 = []
    hlpb3 = []
    hlpb4 = []
    for r in range(len(Sets_50_sorted[0][0][0])):
        if r != 3:
            a = {**ele[0][0][r], **ele[1][0][r]}
            index_bef = list(a.keys())
            hlpb1.append(a)
        else: 
            a = {}
            for ind_i in index_bef:
                for ind_j in index_bef:
                    a[(ind_i,ind_j)] = set_items[3][(ind_i, ind_j)]
            hlpb1.append(a)

    midib.append(hlpb1)

    for r in range(len(Sets_50_sorted[0][0][0])):
        if r != 3:
            a = {**ele[0][0][r], **ele[1][1][r]}
            index_bef = list(a.keys())
            hlpb2.append(a)
        else: 
            a = {}
            for ind_i in index_bef:
                for ind_j in index_bef:
                    a[(ind_i,ind_j)] = set_items[3][(ind_i, ind_j)]
            hlpb2.append(a)

    midib.append(hlpb2)

    for r in range(len(Sets_50_sorted[0][0][0])):
        if r != 3:
            a = {**ele[0][1][r], **ele[1][0][r]}
            index_bef = list(a.keys())
            hlpb3.append(a)
        else: 
            a = {}
            for ind_i in index_bef:
                for ind_j in index_bef:
                    a[(ind_i,ind_j)] = set_items[3][(ind_i, ind_j)]
            hlpb3.append(a)

    midib.append(hlpb3)


    for r in range(len(Sets_50_sorted[0][0][0])):
        if r != 3:
            a = {**ele[0][1][r], **ele[1][1][r]}
            index_bef = list(a.keys())
            hlpb4.append(a)
        else: 
            a = {}
            for ind_i in index_bef:
                for ind_j in index_bef:
                    a[(ind_i,ind_j)] = set_items[3][(ind_i, ind_j)]
            hlpb4.append(a)

    midib.append(hlpb4)

        
    before_model_50.append(midib)

In [46]:
scored_50_4 = []
for ele in merged_sets_50:
    score_4 = []
    for st in ele:
        sc = calculate_score(st)
        score_4.append(sc)
    scored_50_4.append(score_4)

2644
[5, 8, 35, 47, 53, 71, 100, 136, 181, 186, 238, 334, 341, 347, 421, 428, 440, 451, 473, 485, 525, 585, 586, 589, 610, 734, 756, 766, 782, 868, 898, 899, 919, 1030, 1058, 1100, 1122, 1124, 1150, 1161, 1278, 1288, 1300, 1362, 1440, 1469, 2372, 2513, 2596, 2644]
passed
FICO Xpress v9.0.0, Hyper, solve started 20:18:16, Jul 28, 2023
Heap usage: 1687MB (peak 1710MB, 77MB system)
Maximizing MILP noname using up to 8 threads and up to 63GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
      9902 rows      7001315 cols        24792 elements   7001315 entities
Presolved problem has:
      3777 rows         1325 cols        11317 elements      1325 entities
LP relaxation tightened
Presolve finished in 9 seconds
Heap usage: 2543MB (peak 4669MB, 77MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.40e-01,  1.00e+09] / [ 3.05e-05,  1.94e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.81e+05] / [ 

In [47]:
scores_fi = []
for ele in scores_f:
    scores_fi.append({**ele[0], **ele[1]})

# the total 50 scores

In [50]:
scores_finaux = []
sigma_w = []
indices = []
for i in random_index:
    scores_i = []
    sigma_wi = 0
    for ele in Sets_50_sorted:
        if i in ele[0][0][0].keys() or i in ele[1][0][0].keys():
            scores_i.append(scores_fi[Sets_50_sorted.index(ele)][i])
            sigma_wi += 1 
            if i in before_model_50[Sets_50_sorted.index(ele)][0][0].keys():
                scores_i.append(2*scored_50_4[Sets_50_sorted.index(ele)][0][i])
                sigma_wi += 2 
            if i in before_model_50[Sets_50_sorted.index(ele)][1][0].keys():
                scores_i.append(scored_50_4[Sets_50_sorted.index(ele)][1][i])
                sigma_wi += 1 
            if i in before_model_50[Sets_50_sorted.index(ele)][2][0].keys():
                scores_i.append(scored_50_4[Sets_50_sorted.index(ele)][2][i])
                sigma_wi += 1 

        if i in ele[0][1][0].keys() or i in ele[1][1][0].keys():
            scores_i.append(2*scores_fi[Sets_50_sorted.index(ele)][i])
            sigma_wi += 2 
            if i in before_model_50[Sets_50_sorted.index(ele)][1][0].keys():
                scores_i.append(2*scored_50_4[Sets_50_sorted.index(ele)][1][i])
                sigma_wi += 2 
            if i in before_model_50[Sets_50_sorted.index(ele)][2][0].keys():
                scores_i.append(2*scored_50_4[Sets_50_sorted.index(ele)][2][i])
                sigma_wi += 2 
            if i in before_model_50[Sets_50_sorted.index(ele)][3][0].keys():
                scores_i.append(scored_50_4[Sets_50_sorted.index(ele)][3][i]) 
                sigma_wi += 1
    sigma_w.append(sigma_wi)
    indices.append(i)            
    scores_finaux.append((i,scores_i))


In [51]:
scores_dict = {}
sum_scores = [sum(ele[1]) for ele in scores_finaux]
scores_step_9 = list(np.array(sum_scores)/np.array(sigma_w))
scores_dict = {key: value for key, value in zip(indices, scores_step_9)}

# Part II: on sets 100

In [54]:
for st in Set_100:
    index_st = list(st[0].keys())
    vals_st = {}
    for i in index_st:
        vals_st[i] = scores_dict[i]
    st.append(vals_st)

# Ordering the sets

In [63]:
for st in Set_100:
    sorted_items = sorted(st[5].items(), key=lambda item: item[1], reverse=True)
    st[5] = dict(sorted_items)  

for st in Set_100:
    index_by_order = st[5].keys()
    a = {}
    b = {}
    c = {}
    d = {}
    e = {}
    for id in index_by_order:
        a[id] = st[0][id]
        b[id] = st[1][id]
        c[id] = st[2][id]
        for id_j in index_by_order:
            d[(id, id_j)] = st[3][(id, id_j)]
        e[id] = st[4][id]
    st[0] = a
    st[1] = b
    st[2] = c
    st[3] = d
    st[4] = e

# dividing into low/high performing

In [67]:
high_low_Set_100 = []
for st in Set_100: 
    high_low_Set_100.append(divide_set_by_scores(st, 50))

# 10-cycle instances

In [73]:
instance2 = []
instance3 = []
instance4 = []
instance5 = []
high_performing = []
low_performing = []
for st in high_low_Set_100:
    high_performing.append(st[0])
    low_performing.append(st[1])


In [74]:
high_perfo_indices = []
low_perfo_indices = []

for ele in high_performing:
    key = list(ele[0].keys())
    for i in key:
        high_perfo_indices.append(i)

for ele in low_performing:
    key = list(ele[0].keys())
    for i in key:
        low_perfo_indices.append(i)

# forming instances

In [75]:
for i in range(len(high_performing)):
    if i != len(high_performing)-1:
        instance2.append([high_performing[i], high_performing[i+1]])
    else:
        instance2.append([high_performing[len(high_performing)-1], high_performing[0]])

In [76]:
for i in range(len(low_performing)):
    if i != len(low_performing)-1:
        instance3.append([low_performing[i], low_performing[i+1]])
    else:
        instance3.append([low_performing[len(low_performing)-1], low_performing[0]])

In [77]:
for i in range(len(high_performing)):
    if i != len(high_performing)-1:
        instance4.append([high_performing[i],low_performing[i+1]])
    else:
        instance4.append([high_performing[len(high_performing)-1], low_performing[0]])

In [78]:
for i in range(len(high_performing)):
    if i != len(high_performing)-1:
        instance5.append([low_performing[i],high_performing[i+1]])
    else:
        instance5.append([low_performing[len(high_performing)-1], high_performing[0]])

## Making instances ready for the model

In [97]:
instance2_set = []
instance3_set = []
instance4_set = []
instance5_set = []

for ens in instance2:
    two_lis = []
    two_merged = []
    for lis in ens:
        l = lis.copy()
        l.pop()
        two_lis.append(l)
    for i in range(len(two_lis[0])):
        if i != 3:
            a = {**two_lis[0][i],**two_lis[1][i]}
            indices_2 = list(a.keys())
            two_merged.append(a)
        else:
            a = {}
            for ind_i in indices_2:
                for ind_j in indices_2:
                    a[(ind_i, ind_j)] = set_items[3][(ind_i, ind_j)]
            two_merged.append(a)
    instance2_set.append(two_merged)

for ens in instance3:
    two_lis = []
    two_merged = []
    for lis in ens:
        l = lis.copy()
        l.pop()
        two_lis.append(l)
    for i in range(len(two_lis[0])):
        if i != 3:
            a = {**two_lis[0][i],**two_lis[1][i]}
            indices_3 = list(a.keys())
            two_merged.append(a)
        else:
            a = {}
            for ind_i in indices_3:
                for ind_j in indices_3:
                    a[(ind_i, ind_j)] = set_items[3][(ind_i, ind_j)]
            two_merged.append(a)
    instance3_set.append(two_merged)

for ens in instance4:
    two_lis = []
    two_merged = []
    for lis in ens:
        l = lis.copy()
        l.pop()
        two_lis.append(l)
    for i in range(len(two_lis[0])):
        if i != 3:
            a = {**two_lis[0][i],**two_lis[1][i]}
            indices_4 = list(a.keys())
            two_merged.append(a)
        else:
            a = {}
            for ind_i in indices_4:
                for ind_j in indices_4:
                    a[(ind_i, ind_j)] = set_items[3][(ind_i, ind_j)]
            two_merged.append(a)
    instance4_set.append(two_merged)

for ens in instance5:
    two_lis = []
    two_merged = []
    for lis in ens:
        l = lis.copy()
        l.pop()
        two_lis.append(l)
    for i in range(len(two_lis[0])):
        if i != 3:
            a = {**two_lis[0][i],**two_lis[1][i]}
            indices_5 = list(a.keys())
            two_merged.append(a)
        else:
            a = {}
            for ind_i in indices_5:
                for ind_j in indices_5:
                    a[(ind_i, ind_j)] = set_items[3][(ind_i, ind_j)]
            two_merged.append(a)
    instance5_set.append(two_merged)

## Entering the model

In [99]:
scores_instance2 = []
scores_instance3 = []
scores_instance4 = []
scores_instance5 = []

for i in range(len(instance2_set)):
    scores_instance2.append(calculate_score(instance2_set[i]))
    scores_instance3.append(calculate_score(instance3_set[i]))
    scores_instance4.append(calculate_score(instance4_set[i]))
    scores_instance5.append(calculate_score(instance5_set[i]))

2992
[28, 35, 53, 56, 78, 90, 94, 118, 135, 238, 320, 341, 347, 354, 451, 473, 485, 533, 586, 594, 756, 766, 768, 782, 825, 827, 852, 868, 898, 919, 932, 954, 959, 1004, 1030, 1100, 1122, 1150, 1225, 1259, 1278, 1300, 1325, 1362, 1493, 1513, 1540, 1542, 1642, 1652, 1725, 1732, 1777, 1783, 1807, 1813, 1961, 2004, 2020, 2108, 2125, 2205, 2208, 2212, 2249, 2260, 2286, 2290, 2318, 2372, 2373, 2458, 2465, 2472, 2496, 2513, 2514, 2552, 2563, 2575, 2579, 2596, 2617, 2621, 2629, 2644, 2675, 2686, 2700, 2705, 2712, 2736, 2765, 2788, 2790, 2791, 2827, 2850, 2974, 2992]
passed
FICO Xpress v9.0.0, Hyper, solve started 22:20:57, Jul 28, 2023
Heap usage: 2221MB (peak 2221MB, 106MB system)
Maximizing MILP noname using up to 8 threads and up to 63GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
     39802 rows      8964035 cols        99539 elements   8964035 entities
Presolved problem has:
     15052 rows         5150 cols        45089 elements      5150 entities
LP relaxat

## Average score per instance

In [100]:
average_scores2 = {}
average_scores3 = {}
average_scores4 = {}
average_scores5 = {}

for i in random_index:
    scores2_i = []
    for ele in scores_instance2:
        if i in list(ele.keys()):
            scores2_i.append(ele[i])
    if len(scores2_i)>0:
        avg_score_i = sum(scores2_i)/len(scores2_i)
        average_scores2[i] = avg_score_i


for i in random_index:
    scores3_i = []
    for ele in scores_instance3:
        if i in list(ele.keys()):
            scores3_i.append(ele[i])
    if len(scores3_i)>0:
        avg_score_i = sum(scores3_i)/len(scores3_i)
        average_scores3[i] = avg_score_i



for i in random_index:
    scores4_i = []
    for ele in scores_instance4:
        if i in list(ele.keys()):
            scores4_i.append(ele[i])
    if len(scores4_i)>0:
        avg_score_i = sum(scores4_i)/len(scores4_i)
        average_scores4[i] = avg_score_i



for i in random_index:
    scores5_i = []
    for ele in scores_instance5:
        if i in list(ele.keys()):
            scores5_i.append(ele[i])
    if len(scores5_i)>0:
        avg_score_i = sum(scores5_i)/len(scores5_i)
        average_scores5[i] = avg_score_i

## final score per instance

In [101]:
sf_2 = {}
sf_3 = {}
sf_4 = {}
sf_5 = {}

for i in random_index:
    sf_i = []
    for ele in scores_instance2:
        if i in list(ele.keys()):
            items_list = list(ele.items())
            elements_after_i = dict(items_list[i+1:])
            indices_after_i = list(elements_after_i.keys())
            sc_to_max = [average_scores2[j] for j in indices_after_i]
            sc_to_max.append(average_scores2[i])
            max_sc_i = max(sc_to_max)
            sf_i.append(max_sc_i)
    if len(sf_i)>0:
        sf_2[i] = max(sf_i)



for i in random_index:
    sf_i = []
    for ele in scores_instance3:
        if i in list(ele.keys()):
            items_list = list(ele.items())
            elements_after_i = dict(items_list[i+1:])
            indices_after_i = list(elements_after_i.keys())
            sc_to_max = [average_scores3[j] for j in indices_after_i]
            sc_to_max.append(average_scores3[i])
            max_sc_i = max(sc_to_max)
            sf_i.append(max_sc_i)
    if len(sf_i)>0:
        sf_3[i] = max(sf_i)



for i in random_index:
    sf_i = []
    for ele in scores_instance4:
        if i in list(ele.keys()):
            items_list = list(ele.items())
            elements_after_i = dict(items_list[i+1:])
            indices_after_i = list(elements_after_i.keys())
            sc_to_max = [average_scores4[j] for j in indices_after_i]
            sc_to_max.append(average_scores4[i])
            max_sc_i = max(sc_to_max)
            sf_i.append(max_sc_i)
    if len(sf_i)>0:
        sf_4[i] = max(sf_i)



for i in random_index:
    sf_i = []
    for ele in scores_instance5:
        if i in list(ele.keys()):
            items_list = list(ele.items())
            elements_after_i = dict(items_list[i+1:])
            indices_after_i = list(elements_after_i.keys())
            sc_to_max = [average_scores5[j] for j in indices_after_i]
            sc_to_max.append(average_scores5[i])
            max_sc_i = max(sc_to_max)
            sf_i.append(max_sc_i)
    if len(sf_i)>0:
        sf_5[i] = max(sf_i)



# Final scores

In [120]:
scores_final_1000 = []
only_scores = []
for i in random_index:
    scores_i = []
    w_i = []
    if i in high_perfo_indices:
        scores_i.append(2*sf_2[i])
        w_i.append(2)
        scores_i.append(sf_4[i])
        w_i.append(1)       
        scores_i.append(sf_5[i])
        w_i.append(1) 
        scores_i.append(scores_dict[i])
        w_i.append(1)
    if i in low_perfo_indices:
        scores_i.append(sf_3[i])
        w_i.append(1)
        scores_i.append(2*sf_4[i])
        w_i.append(2)       
        scores_i.append(2*sf_5[i])
        w_i.append(2) 
        scores_i.append(2*scores_dict[i])
        w_i.append(2)
    sumi_scores = sum(scores_i)
    sumi_w = sum(w_i)
    if sumi_w > 0:
        sc_fin_100_i = sumi_scores/sumi_w
        scores_final_1000.append((i,sc_fin_100_i))
        only_scores.append(sc_fin_100_i)

# Saving scored data for the ML model

In [133]:
# Define an empty DataFrame with column names
df_scoring = pd.DataFrame(columns=['Product_scoring', 'Volumes_scoring', 'Demand_scoring', 'Costs_scoring', 'Alphas_scoring', 'Price_scoring'])
ind = list(range(1, 1001))

df_scoring['Product_scoring'] = random_index
df_scoring['Volumes_scoring'] = volumes_selec
df_scoring['Demand_scoring'] = demand_selec
df_scoring['Costs_scoring'] = costs_selec
df_scoring['Price_scoring'] = price_selec
df_scoring['Scores'] = only_scores

alpha_li = []
for i in range(1, len(volumes_selec)+1):
    alpha_i = []
    for j in range(1, len(volumes_selec)+1):
        alpha_i.append(alpha[(i,j)])
    alpha_li.append(alpha_i)
df_scoring['Alphas'] = alpha_li
df_scoring.to_csv('data_scoring_1000.csv', index=False)

In [139]:
from IPython.display import FileLink
FileLink('data_scoring_initial.csv')

C:\Users\Abir.HARRASSE\data_scoring_initial.csv